In [ ]:
import tensorflow as tf

import os 

from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import LeakyReLU
from keras.layers import Input
from keras.layers import Dropout
from keras.initializers import RandomNormal


In [7]:
def load(image_file):
    #decode into uint8 tensor
    image = tf.io.read_file(image_file)
    image = tf.io.decode_jpeg(image)

    #split into photo + painting 
    w = tf.shape(image)[1]
    w = w // 2
    input_image = image[:,w:,:]
    real_image = image[:,:w,:]

    #convert both images to float32 tensors 
    input_image = tf.cast(input_image, tf.float32)
    real_image = tf.cast(real_image, tf.float32)

    return input_image, real_image 



In [39]:
def Discriminator(image_shape):

    # stddev can be edited 
    initializer = tf.random_normal_initializer(0.0, 0.05)

    inp_img = Input(shape=image_shape, name='input_image')
    target_img = Input(shape=image_shape, name='target_image')

    merged = Concatenate()([inp_img,target_img])

    # adapted from 'Image-to-Image Translation with Conditional Adversarial Networks' - Isola, UC Berkeley
    # C64-C128-C256-C512
    
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(merged)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(merged)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(merged)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)


    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=initializer)(merged)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)

    # patch output
    patch_out = Activation('sigmoid')(d)
    model = Model([inp_img,target_img], patch_out)
    out = Adam(learning_rate=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', loss_weights=[0.5])

    return model


    

In [ ]:
def encoder(size, n_filters, apply_batchnorm=True):

    init = RandomNormal(stddev=0.02)
    downsample = Conv2D(n_filters, (4,4), strides = (2,2), padding = 'same', kernel_initializer = init)(size)

    if apply_batchnorm:
        downsample = BatchNormalization()(downsample, training=True)

    downsample = LeakyReLU(alpha=0.2)(downsample)
    return downsample


In [ ]:
def decoder(size, skip, n_filters, apply_dropout=False):

    init = RandomNormal(stddev=0.02)
    upsample = Conv2DTranspose(n_filters, (4,4), strides = (2,2), padding = 'same', kernel_initializer = init)(size)
    upsample = BatchNormalization()(upsample, training=True)
    if dropout:
        upsample = Dropout(0.5)(upsample,training=True)
    
    upsample = Concatenate()([upsample,skip])
    upsample = Activation('reLU')(upsample)
    return upsample

In [ ]:
def Generator(image_shape)

    init = RandomNormal(stddev=0.02)
    # adapted from 'Image-to-Image Translation with Conditional Adversarial Networks' - Isola, UC Berkeley
    # C64-C128-C256-C512

    inp_img = Input(shape = image_shape)
    e1 = encoder(inp_img, 64, apply_batchnorm=False)
    e2 = encoder(e1, 128) 
    e3 = encoder(e2, 256) 
    e4 = encoder(e3, 512) 
    e5 = encoder(e4, 512)
    e6 = encoder(e5, 512)
    e7 = encoder(e6, 512)

    bottleneck = Conv2D(512, (4,4), strides = 2, padding = 'same', kernel_initializer=init)(e7)
    bottleneck = Activation('reLU')(bottleneck)

    d1 = decoder(bottleneck, e7,512)
    d2 = decoder(d1, e6, 512)
    d3 = decoder(d2, e5, 512)
    d4 = decoder(d3, e4, 512, dropout=False)
    d5 = decoder(d4, e3, 256, dropout=False)
    d6 = decoder(d5, e2, 128, dropout=False)
    d7 = decoder(d6, e1, 64, dropout=False)

	g = Conv2DTranspose(image_shape[2], (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d7) 
	out_image = Activation('tanh')(g) 

    model = Model(in_image, out_image)
    return model
   